# Converter into OME-Zarr V0.4 (PyMIF 0.2.2)

In [ ]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os
import time

In [ ]:
# --- Set input data path ---
main_path = "../"
dataset_name = "E_for_Nicola"

outpath = "../"
outdataset_name = "cephalopods_ome-zarr_V04"

# main_path = "/g/mif/common/Data/Z_from_LCS_24-07-16/Coll_Montse/MouseEmbryo_micemetyleneBlue/2022-08-09_132304/processed"
# dataset_name = "20221108-162726_Task_2_LM_FullRes_C"

# main_path = "/mif-users/Users/Arias_Adrian/scans_CSL_P21/2400_31573_s"
# dataset_name = "20250523-105843_Task_1_stitching_1_C"

data_path = f"{main_path}/{dataset_name}"

In [ ]:
import glob
flist = glob.glob(f"{data_path}/*.tif")
flist = [f for f in flist if "fusion" in f]
# flist = [f for f in flist if "bkgrnd" not in f]

order = ["-T-", "-F-gfp","-F-Cy5"]
channel_colors = ["FFFFFF", "00FF00","FF00FF"]
channel_names = ["Trans","GFP","Cy5"]
pxl_size = [15.33, 15.33, 15.33]

flist_ord = []
for o in order:
    # print([o in f for f in flist].index(True), flist[[o in f for f in flist].index(True)])
    flist_ord.append(flist[[o in f for f in flist].index(True)])

print(flist_ord)

In [ ]:
from bioio import BioImage
import bioio_tifffile
import numpy as np
img = [BioImage(f, reader=bioio_tifffile.Reader).data for f in flist_ord]
img = np.swapaxes(np.array(img),0,2)[0]

In [ ]:
metadata = {
  "size": [img.shape],
  "scales": [pxl_size],
  "units": ("micrometer","micrometer","micrometer"),
  "axes": "tczyx",
  "channel_names": channel_names,
  "channel_colors": channel_colors,
  # "time_increment": ...,
  # "time_increment_unit": ...,
  # ...
}

In [ ]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: (1,1,16,256,256)
dataset = mm.ArrayManager(array=[img], metadata=metadata, chunks=(1,1,4000,4000,4000))

In [ ]:
# --- Show metadata summary ---
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")
print("CHUNK SIZE:", dataset.chunks)

In [ ]:
dataset.data[0]

In [ ]:
# --- Build pyramid if not already ---
dataset.build_pyramid(
                    num_levels=2, 
                    )

# --- Optional: Write to OME-Zarr format ---
output_zarr_path = f"{outpath}/{outdataset_name}.zarr"
dataset.to_zarr(output_zarr_path)

# Re-read zarr

In [ ]:
output_zarr_path = f"{outpath}/{outdataset_name}.zarr"
zarrdataset = mm.ZarrManager(output_zarr_path, chunks=(1,1,4000,4000,4000), mode="a")

In [ ]:
zarrdataset.create_empty_group("test", metadata=zarrdataset.metadata)

In [ ]:
# --- Show metadata summary for updated dataset ---
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")
print("CHUNK SIZE:", dataset.chunks)

In [ ]:
dataset.close()